In [62]:
import pandas as pd
import yfinance as yf
import webull as wb
import pandas_ta as ta
import numpy as np


In [63]:
# Log into WeBull

email = "raullopez0402@gmail.com"
password = "Gordo2002!"

account = wb.paper_webull()
account.login(email, 'Gordo2002!', "RaulMacBook", save_token=True)
account._access_token = "dc_us_tech1.18cc25ea7d3-aec8be68996b4fe4a446058eb220b63f"


In [64]:
# Check to see if I am logged in
# account.is_logged_in()

In [65]:
# Get Top Cryptos
num_currencies = 1000
crypto_currencies = pd.read_html(f'https://finance.yahoo.com/crypto?offset=0&count={num_currencies}')

In [66]:
# crypto_currencies

In [67]:
# Lets Cache the name of the Cryptos 
symbols = list(crypto_currencies[0]['Symbol'])

crypto_dict = {}

for crypto in symbols:
    # Split name i.e ("BTC-USD") -> ("BTCUSD")
    name, curr = crypto.split("-") 
    stock = name + curr
    
    # See if information is retreivable, if not skip this  
    try:
        df = account.get_bars_crypto(stock=f'{stock}', interval='m5', count=322, extendTrading=0, timeStamp=None)
    except:
        continue
    crypto_dict[stock] = None # Temp

In [68]:
# crypto_dict

In [69]:
from collections import OrderedDict
# Lets create out portfolio
class Portfolio:
    def __init__(self):
        self.balance = 1000000 # Keeps track of our balance
        self.orders = OrderedDict() # Keeps track of our trades 
        

In [70]:
# initiate portfolio
portfolio = Portfolio()

In [71]:
# # Create orderbook
# class OrderBook:
#     def __init__(self):
#         self.book = {}

In [72]:
# # Initiate orderbook
# orderBook = OrderBook()

In [73]:
# Order class
class Order:
    def __init__(self, stock, amount, entry, TP, SL):
        self.stock = stock
        self.amount = amount
        self.entry = entry
        self.take_profit = TP
        self.stop_loss = SL
        

In [74]:
# for crypto in crypto_dict.keys():
#         data = account.get_bars_crypto(stock=f'{crypto}', interval='m1', count=10, extendTrading=0, timeStamp=None)
#         print(data)

In [75]:
# Create a job that checks updates orderBook 
def update_orderBook():
    for crypto in crypto_dict.keys():
        data = account.get_bars_crypto(stock=f'{crypto}', interval='m1', count=322, extendTrading=0, timeStamp=None)
        
        crypto_dict[crypto] = data
    

In [76]:
import joblib

#load the model from disk
loaded_model = joblib.load('myCryptoModel.sav')

In [77]:
from datetime import datetime

def findBuys():
    time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(time)
    for crypto in crypto_dict.keys():
        df = crypto_dict[crypto]
        
        if df == None or df.empty:
            continue
        
        high = list(df['high'])
        low = list(df["low"])
        close = list(df["close"])
        open = list(df["open"])
        volume = list(df["volume"])
        
        dfstream = pd.DataFrame(columns=['Volume','Open','Close','High','Low'])
        
        for i in range(len(df)):
            dfstream.loc[i, ['Volume']] = float(str(volume[i]))
            dfstream.loc[i, ['Open']] = float(str(open[i]))
            dfstream.loc[i, ['Close']] = float(str(close[i]))
            dfstream.loc[i, ['High']] = float(str(high[i]))
            dfstream.loc[i, ['Low']] = float(str(low[i]))
            i=i+1
            
        dfstream['Volume'] = dfstream['Volume'].astype(float)
        dfstream['Open'] = dfstream['Open'].astype(float)
        dfstream['Close'] = dfstream['Close'].astype(float)
        dfstream['High'] = dfstream['High'].astype(float)
        dfstream['Low'] = dfstream['Low'].astype(float)

        dfstream['ATR'] = dfstream.ta.atr(length=20)
        dfstream['RSI'] = dfstream.ta.rsi()
        dfstream['Average'] = (dfstream['High']+dfstream['Low'])/2
        dfstream['MA40'] = dfstream['Open'].rolling(window=40).mean()
        dfstream['MA80'] = dfstream['Open'].rolling(window=80).mean()
        dfstream['MA160'] = dfstream['Open'].rolling(window=160).mean()
        
        backrollingN = 5

        dfstream['MA40Slope'] = dfstream['MA40'].rolling(window = backrollingN).apply(get_slope, raw=True)
        dfstream['MA80Slope'] = dfstream['MA80'].rolling(window = backrollingN).apply(get_slope, raw=True)
        dfstream['MA160Slope'] = dfstream['MA160'].rolling(window = backrollingN).apply(get_slope, raw=True)
        dfstream['AverageSlope'] = dfstream['Average'].rolling(window = backrollingN).apply(get_slope, raw=True)
        dfstream['RSISlope'] = dfstream['RSI'].rolling(window = backrollingN).apply(get_slope, raw=True)

        X_stream = dfstream.iloc[[-1]]

        attributes=['Volume', 'ATR', 'RSI', 'Average', 'MA40', 'MA80', 'MA160', 'MA40Slope', 'MA80Slope', 'MA160Slope', 'AverageSlope', 'RSISlope']
        X_model = X_stream[attributes]
        
        # Apply the model for predictions
        ModelPrediction = loaded_model.predict(X_model)
        
        TPbuy = float(close[i -1]) * 1.03
        SLbuy = float(close[i -1]) * 0.99
        TPSell = float(close[i -1]) * 0.97
        SLSell = float(close[i -1]) * 1.01
        
        if ModelPrediction == 1:
            print("Sell")
            trade_amount = portfolio.balance * 0.01
            quantity = trade_amount // close[i -1]
            
            portfolio.orders[time] = Order(crypto, quantity, close[-1], TPSell, SLSell)
            portfolio.balance -= quantity * close[i -1]
            
            print(f'Sold {quantity} shares of {crypto}, your balance now is {portfolio.balance}.')
        
        elif ModelPrediction == 2:
            trade_amount = portfolio.balance * 0.01
            quantity = trade_amount // close[i -1]
            
            portfolio.orders[time] = Order(crypto, quantity, close[-1], TPBuy, SLBuy)
            portfolio.balance -= quantity * close[i -1]
            
            print(f'Bought {quantity} shares of {crypto}, your balance now is {portfolio.balance}.')

In [78]:
# for _ in portfolio.orders.keys():
#     order = portfolio.orders[_]
        
#     print(order.stock)
    
#     df = crypto_dict[order.stock]
    
#     print(df['close'][])
    
# #     if df[-1] < 1:
# #         print("hit")

In [79]:
from collections import OrderedDict

# Function looks over our portfolio to see if we can sell anything
def findSells():
    temp = OrderedDict()
    
    for _ in portfolio.orders.keys():
        order = portfolio.orders[_]

        df = crypto_dict[order.stock]
        
        # We either reached our Stop Loss or Take Profit mark
        if df['close'][-1] >= order.take_profit or df['close'][-1] < order.stop_loss:
            entry_amount = order.amount * order.entry
            PnL = df['close'][-1] * order.amount # Find the money we made or lost
            portfolio.balance += PnL # Update Balance
            print(f'Made {PnL - entry_amount} dollars on the {order.stock} trade. Our balance now is {portfolio.balance}')
        
        else:
            
            temp[_] = order
        
    portfolio.orders = temp

In [80]:
from scipy.stats import linregress

def get_slope(array):
    y = np.array(array)
    x = np.arange(len(y))
    slope, intercept, r_value, p_value, std_error = linregress(x, y)
    return slope

In [81]:
import schedule
import time

# Define the schedule
schedule.every(15).seconds.do(update_orderBook)
schedule.every(15).seconds.do(findBuys)
schedule.every(61).seconds.do(findSells)

# Run the scheduled tasks forever
while True:
    schedule.run_pending()
    time.sleep(1)  # Sleep for 1 second to avoid excessive CPU usage

2023-12-31 21:14:41


AttributeError: 'NoneType' object has no attribute 'empty'

In [ ]:
print('hello')